In [73]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

##데이터 불러오기

In [74]:
df_origin = pd.read_csv("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv")
df = df_origin
df.head(3)

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0


## EDA

#### 데이터 타입 확인 및 변경

In [75]:
#데이터 확인
df.info()

#sales 값 object에서 float으로 타입 변경
df['NA_Sales'] = pd.to_numeric(df['NA_Sales'],errors = 'coerce') #coerce = 오류값 nan값으로 대체
df['EU_Sales'] = pd.to_numeric(df['EU_Sales'],errors = 'coerce')
df['JP_Sales'] = pd.to_numeric(df['JP_Sales'],errors = 'coerce')
df['Other_Sales'] = pd.to_numeric(df['Other_Sales'],errors = 'coerce')

#Total_sales 칼럼을 새로 만든다.
df['Total_sales'] = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales']

#Total_sales가 0인 행 확인
condition = df["Total_sales"]<=0
print("Total_sales가 0인 행\n",df[condition].count())
print("sales 0값 데이터 제거 전:", df.shape)

#Total Sales가 0인 데이터도 삭제
df.drop(df.loc[df["Total_sales"] == 0].index, inplace = True) 
print("sales 0값 데이터 제거 후:",df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   16598 non-null  int64  
 1   Name         16598 non-null  object 
 2   Platform     16598 non-null  object 
 3   Year         16327 non-null  float64
 4   Genre        16548 non-null  object 
 5   Publisher    16540 non-null  object 
 6   NA_Sales     16598 non-null  object 
 7   EU_Sales     16598 non-null  object 
 8   JP_Sales     16598 non-null  object 
 9   Other_Sales  16598 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.3+ MB
Total_sales가 0인 행
 Unnamed: 0     1
Name           1
Platform       1
Year           1
Genre          1
Publisher      1
NA_Sales       1
EU_Sales       1
JP_Sales       1
Other_Sales    1
Total_sales    1
dtype: int64
sales 0값 데이터 제거 전: (16598, 11)
sales 0값 데이터 제거 후: (16597, 11)


#### 결측치 확인 및 제거

In [76]:
#결측치 확인
print("결측치:\n",df.isnull().sum()) 

print("결측치 제거 전 데이터:",df.shape)

df = df.drop(df.columns[[0]], axis=1) #첫번째 열 제거
df = df.dropna(axis = 0) #결측치가 포함된 모든 행 제거
df = df.reset_index(level=None, drop=True, inplace=False, col_level=0, col_fill='') #인덱스 reset

print("결측치 제거 후 데이터:",df.shape)

결측치:
 Unnamed: 0       0
Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales       200
EU_Sales       200
JP_Sales       199
Other_Sales    200
Total_sales    785
dtype: int64
결측치 제거 전 데이터: (16597, 11)
결측치 제거 후 데이터: (15475, 10)


In [77]:
#duplicated 게임이름 검색하여 확인
print(df.duplicated().sum()) 

#모든 값이 동일한 데이터가 없기 때문에 제거 대상 없음


0


##지역에 따라서 선호하는 게임 장르가 다를까

In [78]:
df[["NA_Sales","EU_Sales","JP_Sales","Other_Sales","Total_sales"]].sum()

NA_Sales       4080.71
EU_Sales       2265.25
JP_Sales       1204.19
Other_Sales     748.17
Total_sales    8298.32
dtype: float64

In [79]:
a = df["NA_Sales"].sum()
b = df["EU_Sales"].sum()
c = df["JP_Sales"].sum()
d = df["Other_Sales"].sum()
e = df["Total_sales"].sum()
print(a,b,c,d,e)
print(a+b+c+d)

4080.71 2265.25 1204.19 748.17 8298.32
8298.32


In [80]:
df.sum()

Name           Candace Kane's Candy FactoryThe MunchablesOtom...
Platform       DSWiiPSPDSPS3DSPCPSPPS3PS3GBAPCDSDSPS3PCPS3PS4...
Year                                                  30862398.0
Genre          ActionActionAdventureMiscPlatformSportsSimulat...
Publisher      DestineerNamco Bandai GamesAlchemistZoo GamesD...
NA_Sales                                                 4080.71
EU_Sales                                                 2265.25
JP_Sales                                                 1204.19
Other_Sales                                               748.17
Total_sales                                              8298.32
dtype: object

In [81]:
condition = df["NA_Sales"] > 0
a = df[condition].value_counts().sum()
df[condition].value_counts()

Name                           Platform  Year    Genre         Publisher        NA_Sales  EU_Sales  JP_Sales  Other_Sales  Total_sales
'98 Koshien                    PS        1998.0  Sports        Magical Company  0.15      0.10      0.12      0.03         0.40           1
Petz Dogz 2                    DS        2007.0  Simulation    Ubisoft          0.47      0.52      0.00      0.12         1.11           1
RPG Maker 3                    PS2       2004.0  Role-Playing  Enterbrain       0.03      0.02      0.00      0.01         0.06           1
RPG Maker II                   PS2       2002.0  Role-Playing  Enterbrain       0.06      0.05      0.00      0.02         0.13           1
RR64: Ridge Racer 64           N64       1999.0  Racing        Nintendo         0.32      0.07      0.00      0.00         0.39           1
                                                                                                                                         ..
Grooverider: Slot Car Thu

In [82]:
condition = df["NA_Sales"] == 0
b = df[condition].value_counts().sum()
df[condition].value_counts()

Name                                               Platform  Year    Genre         Publisher           NA_Sales  EU_Sales  JP_Sales  Other_Sales  Total_sales
.hack//G.U. Vol.1//Rebirth                         PS2       2006.0  Role-Playing  Namco Bandai Games  0.0       0.00      0.17      0.00         0.17           1
Resident Evil 6                                    PC        2012.0  Shooter       Capcom              0.0       0.04      0.00      0.01         0.05           1
Resident Evil Director's Cut                       PS3       2006.0  Action        Capcom              0.0       0.00      0.04      0.00         0.04           1
Resident Evil Director's Cut: Dual Shock Edition   PS        1997.0  Action        Capcom              0.0       0.00      0.16      0.01         0.17           1
Resident Evil: Operation Raccoon City              PC        2012.0  Action        Capcom              0.0       0.03      0.00      0.01         0.04           1
                           

In [83]:
condition = df["NA_Sales"] < 0
c = df[condition].value_counts().sum()

In [84]:
print(a,b,c)
print(a+b+c)

11268 4207 0
15475


##연도별 게임의 트렌드가 있을까

##출고량이 높은 게임에 대한 분석 및 시각화 프로세스

In [85]:
#플렛폼별 게임 갯수 top10
print("플렛폼별 게임 카운트 top10")
print(df['Platform'].value_counts().head(10),"\n")

#플렛폼 별 토탈 게임 출고량 순 top10
print("플렛폼 별 토탈 게임 출고량 순 top10")
print(df.groupby(['Platform']).sum()["Total_sales"].sort_values(ascending = False).head(10))

플렛폼별 게임 카운트 top10
DS      2035
PS2     2024
PS3     1234
Wii     1222
X360    1176
PSP     1137
PS      1121
PC       890
XB       762
GBA      747
Name: Platform, dtype: int64 

플렛폼 별 토탈 게임 출고량 순 top10
Platform
PS2     1172.40
PS3      901.61
X360     895.40
Wii      868.18
DS       787.69
PS       681.18
PSP      282.93
GBA      279.30
PS4      252.97
PC       243.04
Name: Total_sales, dtype: float64


In [86]:
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00,0.04
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01,0.18
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00,0.02
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00,0.04
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04,0.25
...,...,...,...,...,...,...,...,...,...,...
15470,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0.00,0.01,0.20
15471,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0.00,0.00,0.00,0.01
15472,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13,0.79
15473,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03,0.38
